## Remove waves from the historical record that are associated to historical TCs


inputs required:
  * Wave partitions (GOW1 time series)
  * Wave sectors that influence the site
  * Historical TC parameters that affected the site (output of notebook 03)
  

In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

# common
import os
import os.path as op
from datetime import datetime

# pip
import numpy as np
import xarray as xr

# DEV: override installed teslakit
import sys
sys.path.insert(0, op.join(os.path.abspath(''), '..', '..'))

# teslakit
from teslakit.database import Database
from teslakit.waves import GetDistribution



## Database and Site parameters

In [2]:
# --------------------------------------
# Teslakit database

p_data = r'/Users/albacid/Projects/TeslaKit_projects'
db = Database(p_data)

# set site
db.SetSite('GUAM')


In [3]:
# --------------------------------------
# load data and set parameters

xds_wvs_pts = db.Load_WAVES_partitions()  # waves partitions data
_, xds_TCs_r2_params = db.Load_TCs_r2()   # TCs historical parameters inside radius 2

# wave families sectors
wvs_sectors = [(200, 45), (45, 145)]

# date limits for TCs removal from waves data, and TC time window (hours)
tc_rm_date1 = '1979-01-01'
tc_rm_date2 = '2015-12-31'
tc_time_window = 12



## Calculate Waves Families

In [5]:
# --------------------------------------
# Calculate wave families from waves partitions data and waves sectors

xds_wvs_fam = GetDistribution(xds_wvs_pts, wvs_sectors)

print(xds_wvs_pts)
print()
print(xds_wvs_fam)


<xarray.Dataset>
Dimensions:  (time: 111040)
Coordinates:
  * time     (time) datetime64[ns] 1979-01-01 ... 2016-12-31T21:00:00
Data variables:
    hs       (time) float64 2.079 2.134 2.085 2.081 ... 2.159 2.091 2.032 1.932
    phs0     (time) float64 0.1441 0.19 0.168 0.1509 ... 0.1403 0.1241 0.1128
    phs1     (time) float64 1.716 1.368 1.278 1.313 ... 1.529 1.422 1.424 1.418
    phs2     (time) float64 1.139 1.182 1.2 1.244 1.165 ... 1.4 1.4 1.315 1.284
    phs3     (time) float64 0.2206 1.088 1.09 0.9852 ... 0.5605 0.5425 0.2379
    phs4     (time) float64 0.05664 0.2488 0.2187 0.22 ... 0.2324 0.228 0.05955
    phs5     (time) float64 0.05393 0.05729 0.06455 0.08161 ... nan 0.06576 nan
    tp       (time) float64 9.356 9.379 9.433 9.499 ... 9.645 9.404 9.335 9.435
    ptp0     (time) float64 3.213 2.962 4.8 4.335 ... 5.162 3.671 3.346 3.512
    ptp1     (time) float64 8.925 8.998 8.823 5.283 ... 6.021 7.148 9.383 9.521
    ptp2     (time) float64 9.989 5.583 5.285 9.333 ... 9.649 

In [6]:
# --------------------------------------
# Remove TCs from WAVES data (partitions and families)

# remove TCs before 1979 and after 2015
d1 = np.datetime64(tc_rm_date1)
d2 = np.datetime64(tc_rm_date2)
dmin_dates = xds_TCs_r2_params.dmin_date.values

p1 = np.where(dmin_dates >= d1)[0][0]
p2 = np.where(dmin_dates <= d2)[0][-1]
xds_TCs_r2_params = xds_TCs_r2_params.sel(storm = slice(p1, p2))

# storms inside circle
storms = xds_TCs_r2_params.storm.values[:]

# for each storm: find hs_max instant and clean waves data inside "hs_max window"
xds_wvs_pts_noTCs = xds_wvs_pts.copy()
xds_wvs_fam_noTCs = xds_wvs_fam.copy()

# store removed waves (TCs window) families inside a storm category dictionary
d_wvs_fam_cats = {'{0}'.format(k):[] for k in range(6)}

for s in storms:

    # storm dates at dist_min and storm_end
    xds_s = xds_TCs_r2_params.sel(storm = slice(s, s))
    date_dmin = xds_s.dmin_date.values[0]
    date_last = xds_s.last_date.values[0]
    cat = xds_s.category.values[0]

    xds_wvs_s = xds_wvs_pts.sel(
        time = slice(str(date_dmin), str(date_last))
    )

    # get hs_max date 
    t_hs_max = xds_wvs_s.where(
        xds_wvs_s.hs ==  xds_wvs_s.hs.max(), drop=True
    ).time.values[:][0]

    # hs_max time window 
    w1 = t_hs_max - np.timedelta64(tc_time_window,'h')
    w2 = t_hs_max + np.timedelta64(tc_time_window,'h')

    # clean waves partitions 
    xds_wvs_pts_noTCs = xds_wvs_pts_noTCs.where(
        (xds_wvs_pts_noTCs.time < w1) |
        (xds_wvs_pts_noTCs.time > w2)
    )
    # clean waves families
    xds_wvs_fam_noTCs = xds_wvs_fam_noTCs.where( 
        (xds_wvs_fam_noTCs.time < w1) |
        (xds_wvs_fam_noTCs.time > w2)
    )
    
    # store waves families inside storm-removal-window
    xds_fams_s = xds_wvs_fam.sel(time = slice(w1, w2))
    d_wvs_fam_cats['{0}'.format(cat)].append(xds_fams_s)
    
# merge removal-window waves families data
for k in d_wvs_fam_cats.keys():
    d_wvs_fam_cats[k] = xr.merge(d_wvs_fam_cats[k])
    
print('\nWaves Partitions (TCs removed) ')
print(xds_wvs_pts_noTCs)
print('\nWaves Families   (TCs removed) ')
print(xds_wvs_fam_noTCs)    
    
# store results
db.Save_WAVES_ptsfams_noTCs(xds_wvs_pts_noTCs, xds_wvs_fam_noTCs)

# store waves families data (TCs removal window)
db.Save_WAVES_fams_TCs_categ(d_wvs_fam_cats)



Waves Partitions (TCs removed) 
<xarray.Dataset>
Dimensions:  (time: 111040)
Coordinates:
  * time     (time) datetime64[ns] 1979-01-01 ... 2016-12-31T21:00:00
Data variables:
    hs       (time) float64 2.079 2.134 2.085 2.081 ... 2.159 2.091 2.032 1.932
    phs0     (time) float64 0.1441 0.19 0.168 0.1509 ... 0.1403 0.1241 0.1128
    phs1     (time) float64 1.716 1.368 1.278 1.313 ... 1.529 1.422 1.424 1.418
    phs2     (time) float64 1.139 1.182 1.2 1.244 1.165 ... 1.4 1.4 1.315 1.284
    phs3     (time) float64 0.2206 1.088 1.09 0.9852 ... 0.5605 0.5425 0.2379
    phs4     (time) float64 0.05664 0.2488 0.2187 0.22 ... 0.2324 0.228 0.05955
    phs5     (time) float64 0.05393 0.05729 0.06455 0.08161 ... nan 0.06576 nan
    tp       (time) float64 9.356 9.379 9.433 9.499 ... 9.645 9.404 9.335 9.435
    ptp0     (time) float64 3.213 2.962 4.8 4.335 ... 5.162 3.671 3.346 3.512
    ptp1     (time) float64 8.925 8.998 8.823 5.283 ... 6.021 7.148 9.383 9.521
    ptp2     (time) float64 9